In [1]:
# import required libriaries
from pathlib import Path
from helpers import *

In [2]:
# path to data directory
#path = Path('/home/jupyter/fpl-prediction/data')
path = Path('./data')

In [3]:
df_train = pd.read_csv(path/'train_v5.csv', index_col=0, dtype={'season':str})

In [4]:
# read in latest training set
df_train = pd.read_csv(path/'train_v5.csv', index_col=0, dtype={'season':str})

# path to current season
season_paths = [path/'2020-21']

# name for current season
season_names = ['2021']

# team codes
teams = pd.read_csv(path/'teams.csv')

# latest remaining season dataset, for latest week's play_proba values
remaining_season = pd.read_csv(path/'remaining_season.csv', index_col=0)

# create players dataset
all_players = build_players(path, season_paths, season_names, teams)

# 2019-20 market value
# table was in slightly different position for 2019-20 season
# should check that 2019-20 season values remain the same once season starts
teams_mv = build_season_mv('2021', 11, range(13,33))

In [5]:
all_players

,full_name,play_proba_2021,position_2021,id_2021,cost_2021,team_2021,web_name_2021
0,Mesut_Özil,0,3,1,68,3,Özil
1,Sokratis_Papastathopoulos,0,2,2,48,3,Sokratis
2,David_Luiz Moreira Marinho,100,2,3,54,3,David Luiz
3,Pierre-Emerick_Aubameyang,75,3,4,114,3,Aubameyang
4,Cédric_Soares,100,2,5,46,3,Cédric
...,...,...,...,...,...,...,...
606,Meritan_Shabani,None,3,553,45,39,Shabani
607,Rayan_Ait Nouri,100,2,563,50,39,Ait Nouri
608,Owen_Otasowie,None,3,581,45,39,Otasowie
609,Theo_Corbeanu,None,3,610,45,39,Corbeanu


In [6]:
df_train.tail(50)

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season,play_proba,relative_market_value_team_season,relative_market_value_opponent_team_season
97797,Tariq_Lamptey,13,2,77,Brighton and Hove Albion,Fulham,0.534289,0.486494,False,6,...,2.0,-45120,12994,58114,0,2020-12-16T20:00:00Z,2021,0.75,NaN,NaN
97798,Teden_Mengi,13,2,0,Manchester United,Sheffield United,1.696657,0.387306,False,0,...,0.0,3797,7077,3280,0,2020-12-17T20:00:00Z,2021,1.00,NaN,NaN
97799,Terence_Kongolo,13,2,0,Fulham,Brighton and Hove Albion,0.486494,0.534289,True,0,...,0.0,-56,5,61,0,2020-12-16T20:00:00Z,2021,0.25,NaN,NaN
97800,Theo_Walcott,13,3,62,Southampton,Arsenal,0.535526,1.317898,False,9,...,38.0,-4816,4778,9594,0,2020-12-16T18:00:00Z,2021,1.00,NaN,NaN
97801,Thiago_Alcántara do Nascimento,13,3,0,Liverpool,Tottenham Hotspur,2.474081,1.670117,True,0,...,0.0,-2919,206,3125,0,2020-12-16T20:00:00Z,2021,0.00,NaN,NaN
97802,Thiago_Thiago,13,2,90,Chelsea,Wolverhampton Wanderers,1.860172,0.911137,False,1,...,2.0,-3359,7505,10864,0,2020-12-15T18:00:00Z,2021,1.00,NaN,NaN
97803,Thomas_Partey,13,3,0,Arsenal,Southampton,1.317898,0.535526,True,0,...,0.0,-4174,102,4276,0,2020-12-16T18:00:00Z,2021,0.00,NaN,NaN
97804,Timothy_Castagne,13,2,0,Leicester City,Everton,1.091969,1.137628,True,0,...,0.0,-38303,180,38483,0,2020-12-16T18:00:00Z,2021,0.25,NaN,NaN
97805,Timothy_Fosu-Mensah,13,2,0,Manchester United,Sheffield United,1.696657,0.387306,False,0,...,0.0,-69,57,126,0,2020-12-17T20:00:00Z,2021,1.00,NaN,NaN
97806,Timo_Werner,13,4,90,Chelsea,Wolverhampton Wanderers,1.860172,0.911137,False,2,...,50.0,-138035,18781,156816,0,2020-12-15T18:00:00Z,2021,1.00,NaN,NaN


In [7]:
# find the latest gameweek
last_gw = df_train['gw'][df_train['season'] == season_names[0]].max()

if np.isnan(last_gw): 
    last_gw = 1 
else: 
    last_gw = last_gw + 1
    
last_gw

14

In [8]:
# build training set for latest gameweek
df_latest = build_season(season_paths[0], season_names[0], all_players, teams, teams_mv, [last_gw])

In [9]:
df_latest.drop_duplicates(inplace=True)

In [10]:
df_latest.head(50)

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,selected,team_a_score,team_h_score,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season
0,Aaron_Connolly,14,4,90,Brighton and Hove Albion,Sheffield United,0.534079,0.387154,True,2,...,32454,1,1,34.0,-1589,979,2568,0,2020-12-20T12:00:00Z,2021
1,Aaron_Cresswell,14,2,90,West Ham United,Chelsea,0.663130,1.859440,False,1,...,891542,0,3,8.0,5430,37935,32505,0,2020-12-21T20:00:00Z,2021
2,Aaron_Mooy,14,3,0,Brighton and Hove Albion,Sheffield United,0.534079,0.387154,True,0,...,897,1,1,0.0,-4,1,5,0,2020-12-20T12:00:00Z,2021
3,Aaron_Ramsdale,14,1,90,Sheffield United,Brighton and Hove Albion,0.387154,0.534079,False,2,...,173530,1,1,0.0,-6437,1604,8041,1,2020-12-20T12:00:00Z,2021
4,Aaron_Wan-Bissaka,14,2,90,Manchester United,Leeds,1.695990,0.439606,True,1,...,650952,2,6,0.0,-11740,11590,23330,0,2020-12-20T16:30:00Z,2021
5,Abdoulaye_Doucouré,14,3,90,Everton,Arsenal,1.145049,1.317380,True,2,...,79540,1,2,0.0,-1091,3113,4204,0,2020-12-19T17:30:00Z,2021
6,Aboubakar_Kamara,14,3,0,Fulham,Newcastle United,0.486303,0.602651,False,0,...,3703,1,1,0.0,-24,58,82,0,2020-12-19T20:00:00Z,2021
7,Adama_Traoré,14,3,30,Wolverhampton Wanderers,Burnley,0.910778,0.341918,False,1,...,452687,1,2,0.0,-10732,4815,15547,0,2020-12-21T17:30:00Z,2021
8,Adam_Forshaw,14,3,0,Leeds,Manchester United,0.439606,1.695990,False,0,...,580,2,6,0.0,-3,8,11,0,2020-12-20T16:30:00Z,2021
9,Adam_Lallana,14,3,90,Brighton and Hove Albion,Sheffield United,0.534079,0.387154,True,2,...,24628,1,1,14.0,1063,1526,463,0,2020-12-20T12:00:00Z,2021


In [11]:
df_latest.shape

(611, 34)

In [12]:
last_play_proba = remaining_season[remaining_season['gw'] == last_gw][['player', 'play_proba']]

In [13]:
last_play_proba.head()

,player,play_proba
89,Glenn_Murray,0.0
90,Adam_Lallana,1.0
91,Pascal_Groß,1.0
92,Shane_Duffy,0.0
93,Davy_Pröpper,1.0


In [14]:
# add latest week's play_proba for each player
df_latest = df_latest.merge(last_play_proba, on='player', how='left')

In [15]:
df_latest.drop_duplicates(inplace=True)

In [16]:
df_train.drop_duplicates(inplace=True)

In [17]:
if last_gw==1:
    df_latest['relative_market_value_team_season'] = df_latest['relative_market_value_team']
    df_latest['relative_market_value_opponent_team_season'] = df_latest['relative_market_value_opponent_team']

In [18]:
df_train.shape

(97847, 37)

In [19]:
df_latest.shape

(611, 35)

In [20]:
df_train_new = pd.concat([df_train, df_latest], ignore_index=True, axis=0, sort=False)

In [21]:
df_train_new.tail()

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season,play_proba,relative_market_value_team_season,relative_market_value_opponent_team_season
98453,Yves_Bissouma,14,3,90,Brighton and Hove Albion,Sheffield United,0.534079,0.387154,True,2,...,8.0,-4293,6001,10294,0,2020-12-20T12:00:00Z,2021,1.0,NaN,NaN
98454,Zack_Steffen,14,1,0,Manchester City,Southampton,2.383177,0.535315,False,0,...,0.0,-8,35,43,0,2020-12-19T15:00:00Z,2021,1.0,NaN,NaN
98455,Zeze Steven_Sessegnon,14,2,0,Fulham,Newcastle United,0.486303,0.602651,False,0,...,0.0,-34,16,50,0,2020-12-19T20:00:00Z,2021,0.0,NaN,NaN
98456,Çaglar_Söyüncü,14,2,0,Leicester City,Tottenham Hotspur,1.091540,1.669460,False,0,...,0.0,-4435,17,4452,0,2020-12-20T14:15:00Z,2021,0.0,NaN,NaN
98457,Ørjan_Nyland,14,1,0,Aston Villa,West Bromwich Albion,0.733614,0.255967,False,0,...,0.0,-14973,57,15030,0,2020-12-20T19:15:00Z,2021,0.0,NaN,NaN


In [22]:
df_train_new.shape

(98458, 37)

In [23]:
# save latest training set to csv
# overwrite existing one
df_train_new.to_csv(path/'train_v5.csv')

In [24]:
# df_train_new = pd.read_csv(path/'train_v5.csv', index_col=0, dtype={'season':str})

In [25]:
# df_train_new[(df_train_new['team'] == 'Manchester United') & 
#              (df_train_new['gw'] == 32) & 
#              (df_train_new['season'] == '1920')]

In [26]:
## now need to create the prediction set
# start by reading fixtures.csv
# set starting gameweek (where are we right now in the season)

# use these two lines if updating play_proba later in week
# current_gw = last_gw
# df_train_new = pd.read_csv(path/'train_v5.csv', index_col=0, dtype={'season':str})

current_gw = last_gw + 1
fixtures = pd.read_csv(path/'fixtures.csv')
fixtures = fixtures[fixtures['gw'] >= current_gw]

In [27]:
fixtures.head(20)

,home_team,away_team,gw
8,Burnley,Manchester United,28
9,Manchester City,Aston Villa,28
100,Aston Villa,Newcastle United,28
140,Arsenal,Chelsea,15
141,Aston Villa,Crystal Palace,15
142,Fulham,Southampton,15
143,Leeds,Burnley,15
144,Leicester City,Manchester United,15
145,Liverpool,West Bromwich Albion,15
146,Manchester City,Newcastle United,15


In [28]:
# add team codes for home and away teams
fixtures = fixtures.merge(teams, left_on='home_team', right_on='team', how='left')
fixtures = fixtures.merge(teams, left_on='away_team', right_on='team', how='left')
fixtures = fixtures[['gw', 'home_team', 'away_team', 'team_code_x', 'team_code_y']]
fixtures.rename(index=str,
                columns={'team_code_x':'home_team_code',
                         'team_code_y':'away_team_code'},
                inplace=True)

In [29]:
fixtures.head()

,gw,home_team,away_team,home_team_code,away_team_code
0,28,Burnley,Manchester United,90,1
1,28,Manchester City,Aston Villa,43,7
2,28,Aston Villa,Newcastle United,7,4
3,15,Arsenal,Chelsea,3,8
4,15,Aston Villa,Crystal Palace,7,31


In [30]:
# all_players.replace(to_replace='None', value=1, inplace=True)

In [31]:
all_players[all_players['play_proba_2021'] == '75']

,full_name,play_proba_2021,position_2021,id_2021,cost_2021,team_2021,web_name_2021
3,Pierre-Emerick_Aubameyang,75,3,4,114,3,Aubameyang
24,Gabriel Teodoro_Martinelli Silva,75,3,26,49,3,Martinelli
52,Ezri_Konsa Ngoyo,75,2,46,48,7,Konsa
59,Ross_Barkley,75,3,106,59,7,Barkley
106,Johann Berg_Gudmundsson,75,3,90,54,90,Gudmundsson
109,Matej_Vydra,75,4,93,48,90,Vydra
111,Robbie_Brady,75,3,95,50,90,Brady
116,Dwight_McNeil,75,3,100,57,90,McNeil
229,Ivan Ricardo_Neves Abreu Cavaleiro,75,3,187,53,54,Cavaleiro
387,Scott_McTominay,75,3,309,49,1,McTominay


In [33]:
# join home team to all players for current season
home_df = fixtures.merge(all_players, 
               left_on='home_team_code', 
               right_on='team_2021', 
               how='left')

# pull out the required fields and rename columns
home_df = home_df[['gw', 'home_team', 'away_team', 'full_name', 'position_2021', 
                   'cost_2021', 'play_proba_2021', 'web_name_2021']]
home_df.rename(index=str, 
               columns={'home_team':'team',
                        'away_team':'opponent_team',
                        'full_name':'player',
                        'position_2021':'position',
                        'cost_2021':'price',
                        'play_proba_2021':'play_proba',
                        'web_name_2021':'short_name'},
              inplace=True)

# add home flag
home_df['was_home'] = True

In [34]:
# join away team to all players for current season
away_df = fixtures.merge(all_players, 
               left_on='away_team_code', 
               right_on='team_2021', 
               how='left')

# pull out the required fields and rename columns
away_df = away_df[['gw',  'away_team', 'home_team', 'full_name', 'position_2021', 
                   'cost_2021', 'play_proba_2021', 'web_name_2021']]
away_df.rename(index=str, 
               columns={'away_team':'team',
                        'home_team':'opponent_team',
                        'full_name':'player',
                        'position_2021':'position',
                        'cost_2021':'price',
                        'play_proba_2021':'play_proba',
                        'web_name_2021':'short_name'},
              inplace=True)

# add home flag
away_df['was_home'] = False

In [35]:
# look at away players
away_df.head()

,gw,team,opponent_team,player,position,price,play_proba,short_name,was_home
0,28,Manchester United,Burnley,Sergio_Romero,1,49,0,Romero,False
1,28,Manchester United,Burnley,Juan_Mata,3,59,None,Mata,False
2,28,Manchester United,Burnley,David_de Gea,1,53,100,de Gea,False
3,28,Manchester United,Burnley,Chris_Smalling,2,49,0,Smalling,False
4,28,Manchester United,Burnley,Odion_Ighalo,4,61,None,Ighalo,False


In [36]:
# look at home players
home_df.head()

,gw,team,opponent_team,player,position,price,play_proba,short_name,was_home
0,28,Burnley,Manchester United,Dale_Stephens,3,43,100,Stephens,True
1,28,Burnley,Manchester United,James_Tarkowski,2,53,100,Tarkowski,True
2,28,Burnley,Manchester United,Phil_Bardsley,2,45,100,Bardsley,True
3,28,Burnley,Manchester United,Erik_Pieters,2,43,100,Pieters,True
4,28,Burnley,Manchester United,Jack_Cork,3,50,0,Cork,True


In [37]:
# calculate average minutes over last 5 gameweeks
# first get last 5 gameweeks in current season
recent_mins = df_train_new[(df_train_new['gw'] >= current_gw - 5) & 
                           (df_train_new['season'] == season_names[0])][['player', 'minutes', 'play_proba']]

# recent_mins = df_train_new[((df_train_new['gw'] >= 40 - 5) & 
#                            (df_train_new['season'] == '1920')) | 
#                            ((df_train_new['gw'] >= 1) & 
#                            (df_train_new['season'] == '2021'))][['player', 'minutes', 'play_proba']]

# exclude weeks where the player had an injury designation (i.e. keep where play_proba is 1)
recent_mins = recent_mins[recent_mins['play_proba'] == 1]

len(recent_mins)

# average for each player
recent_mins = recent_mins.groupby(['player'])['minutes'].mean().to_dict()

recent_mins

{'Aaron_Connolly': 48.6,
 'Aaron_Cresswell': 90.0,
 'Aaron_Ramsdale': 90.0,
 'Aaron_Wan-Bissaka': 90.0,
 'Abdoulaye_Doucouré': 88.0,
 'Aboubakar_Kamara': 12.6,
 'Adam_Lallana': 90.0,
 'Adam_Webster': 72.0,
 'Adama_Traoré': 65.8,
 'Ademola_Lookman': 89.2,
 'Adrián_Bernabé': 0.0,
 'Adrián_San Miguel del Castillo': 0.0,
 'Ahmed_El Mohamady': 23.0,
 'Ainsley_Maitland-Niles': 39.2,
 'Aleksandar_Mitrović': 19.75,
 'Alex Nicolao_Telles': 50.75,
 'Alex_Iwobi': 82.6,
 'Alex_McCarthy': 90.0,
 'Alex_Oxlade-Chamberlain': 7.5,
 'Alexandre_Lacazette': 34.4,
 'Alexis_Mac Allister': 0.0,
 'Alireza_Jahanbakhsh': 27.8,
 'Alisson_Ramses Becker': 67.5,
 'Allan_Marques Loureiro': 77.0,
 'Allan_Saint-Maximin': 0.0,
 'Alphonse_Areola': 90.0,
 'Andi_Zeqiri': 3.6,
 'Andreas_Christensen': 0.0,
 'Andrew_Robertson': 90.0,
 'Andriy_Yarmolenko': 3.3333333333333335,
 'Andros_Townsend': 30.2,
 'André Filipe_Tavares Gomes': 28.0,
 'André-Frank_Zambo Anguissa': 90.0,
 'Andy_Carroll': 0.0,
 'Angelo_Ogbonna': 90.0,
 'Ant

In [38]:
# concatenate home and away players
remaining_season_df = home_df.append(away_df).reset_index(drop=True)

# add home and away team market values
remaining_season_df = remaining_season_df.merge(teams_mv[teams_mv['season'] == '2021'],
                                                left_on='team', 
                                                right_on='name', 
                                                how='left').drop(['name', 'season'], axis=1)

remaining_season_df = remaining_season_df.merge(teams_mv[teams_mv['season'] == '2021'],
                                                left_on='opponent_team', 
                                                right_on='name', 
                                                how='left').drop(['name', 'season'], axis=1)

remaining_season_df.rename(index=str, 
                           columns={'relative_market_value_x':'relative_market_value_team',
                                    'relative_market_value_y':'relative_market_value_opponent_team'},
                           inplace=True)

# add season name
remaining_season_df['season'] = season_names[0]

# divide cost by 10 for actual cost
remaining_season_df['price'] = remaining_season_df['price']/10

# set availability probability
# 0 = 0% chance, 25 = 25% chance, etc
# 'None' or '100' = 100% chance
remaining_season_df.loc[remaining_season_df['play_proba'] == 'None', 'play_proba'] = 100
remaining_season_df['play_proba'] = remaining_season_df['play_proba'].astype('float') / 100

# cast position to integer
remaining_season_df['position'] = remaining_season_df['position'].astype(int)

# add minutes based on last five fixtures
remaining_season_df['minutes'] = remaining_season_df['player'].map(recent_mins)
remaining_season_df['minutes'].fillna(0, inplace=True)

# multiply minutes by play probability
remaining_season_df['minutes'] = remaining_season_df['minutes'] * remaining_season_df['play_proba']

In [39]:
remaining_season_df.head(50)

,gw,team,opponent_team,player,position,price,play_proba,short_name,was_home,relative_market_value_team,relative_market_value_opponent_team,season,minutes
0,28,Burnley,Manchester United,Dale_Stephens,3,4.3,1.00,Stephens,True,0.341918,1.695990,2021,0.00
1,28,Burnley,Manchester United,James_Tarkowski,2,5.3,1.00,Tarkowski,True,0.341918,1.695990,2021,90.00
2,28,Burnley,Manchester United,Phil_Bardsley,2,4.5,1.00,Bardsley,True,0.341918,1.695990,2021,0.00
3,28,Burnley,Manchester United,Erik_Pieters,2,4.3,1.00,Pieters,True,0.341918,1.695990,2021,9.00
4,28,Burnley,Manchester United,Jack_Cork,3,5.0,0.00,Cork,True,0.341918,1.695990,2021,0.00
5,28,Burnley,Manchester United,Kevin_Long,2,4.4,1.00,Long,True,0.341918,1.695990,2021,0.00
6,28,Burnley,Manchester United,Jay_Rodriguez,4,5.8,1.00,Rodriguez,True,0.341918,1.695990,2021,58.80
7,28,Burnley,Manchester United,Ashley_Barnes,4,6.0,1.00,Barnes,True,0.341918,1.695990,2021,46.40
8,28,Burnley,Manchester United,Ben_Mee,2,4.9,1.00,Mee,True,0.341918,1.695990,2021,90.00
9,28,Burnley,Manchester United,Ashley_Westwood,3,5.3,1.00,Westwood,True,0.341918,1.695990,2021,90.00


In [40]:
# save latest prediction set to csv
remaining_season_df.to_csv(path/'remaining_season.csv')